# Importing Packages

In [5]:
import sys
import os
import pandas as pd
from Bio import SeqIO, Seq
import shutil
import gzip
import subprocess

# Defining Variables

In [6]:
appRoot = "/Users/agolfinos/Desktop/SIV_Transmission_Paper/Sequences_IR/out/"


# Converting fastq.gz to .fastq

In [47]:
for file in files: 
    #ignoring hidden files or other things in the directory
    if file.endswith(".fastq.gz"):
        #saving the file name as a string
        name = str(file)
        fname = str(file)[0:-3]
        
    with gzip.open(dir + "/" + name, 'rb') as f_in:
        with open(dir + "/" + fname, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)
        


r04103_Week10-R2-24194.fastq
r10001_Week8-R2-24176.fastq
r04103_Day14-R1-24189.fastq
r10001_Day21-R2-24173.fastq
r10001_Week12(necropsy)-R1-24170.fastq
r04103_Week6-R1-24192.fastq
r04103_Week12(necropsy)-R2-24195.fastq
r10001_Day14-R2-24164.fastq
r10001_Week6-R2-24175.fastq
r04103_Day7-R2-24188.fastq
r10001_Day7-R2-24163.fastq
r10001_Week10-R1-24177.fastq
r04103_Day28-R1-24191.fastq
r04103_Day21-R1-24182.fastq
r10001_Day21-R2-24165.fastq
r10001_Day14-R2-24172.fastq
r10001_Day28-R1-24166.fastq
r04103_Week6-R1-24184.fastq
r04103_Day14-R2-24189.fastq
r10001_Week8-R1-24176.fastq
r04103_Week10-R1-24194.fastq
r10001_Day14-R1-24164.fastq
r10001_Week6-R1-24175.fastq
r04103_Week6-R2-24192.fastq
r04103_Week12(necropsy)-R1-24195.fastq
r10001_Week12(necropsy)-R2-24170.fastq
r10001_Day21-R1-24173.fastq
r10001_Day7-R1-24163.fastq
r04103_Day7-R1-24188.fastq
r10001_Day28-R2-24166.fastq
r04103_Week6-R2-24184.fastq
r10001_Day14-R1-24172.fastq
r10001_Day21-R1-24165.fastq
r10001_Week10-R2-24177.fastq
r041

# Convert .fastq to .fasta

In [53]:
#updating the file list from the directory
files = os.listdir(dir)

for file in files: 
    #ignoring hidden files or other things in the directory
    if file.endswith(".fastq"):
        #saving the file name as a string
        name = str(file)
        #saving the file name without the .fastq extension
        fname = str(file)[0:-6]
        #creating the file name for the fasta that we are going to output
        outfname = fname + ".fasta"

        #dictates the name of the input file as a read only file
        with open(dir + "/" + name, "r") as input_handle: 
            #dictates the name of the output file as a write-in file
            with open(dir + "/" + outfname, "w") as output_handle: 
                #parses the input fastq file
                sequences = SeqIO.parse(input_handle, "fastq")
                #writes the parts from the previous statement to the output file as a fasta instead of a fastq
                count = SeqIO.write(sequences, output_handle, "fasta")

# Merging Reads

In [66]:
#reference file for mapping
ref1 = appRoot + "/ref/M33262.fasta"

#first file name
fname1 = "r10001_Week12necropsy-R1-24178"

fname2 = "r10001_Week12necropsy-R2-24178"

outputf = "r10001_Week12_Rep2"

#first paired end fastq file (NEEDS TO HAVE QUALITY INFO TO MERGE!!)
pe1 = appRoot +  '/UnmappedUntrimmedUnmerged/' + fname1 + ".fastq"

#second paired end fastq file
pe2 = appRoot + '/UnmappedUntrimmedUnmerged/' + fname2 + ".fastq"

#output file that contains mapped and trimmed reads
out = appRoot + "/out/" + outputf + "merged_reads.fastq"

##################MERGING#############
merge_cmd = [appRoot + '/bbmap/bbmerge.sh', "in=" + pe1, "in2=" + pe2, "out=" + out, "qtrim=t",
            "trimq=20"]
subprocess.call(merge_cmd)

0

# Mapping Reads

In [110]:
#folder absolute path to all files used in this tool
appRoot = "/Users/agolfinos/Desktop/SIV_Transmission_Paper/Sequences_IR/out"

#reference file for mapping
ref1 = appRoot + "/ref/M33262.fasta"

#animal number
anNum = "r10001"

#timepoint
timePT = "Week12"

#Rep#
rep = "2"

#merged file name
filename = anNum + "_" + timePT + "_Rep" + rep

#name and absolute path to the merged file we will use for mapping
pe1 = appRoot + "/" + filename + "merged_reads.fastq"

#output file that contains mapped reads
out = appRoot + "/" + filename + "merged_mapped_reads.fastq"
    
#mapping reads to the reference without writing an index
map_cmd = [appRoot + '/bbmap/bbmap.sh', "ref=" + ref1, "in=" + pe1,  
          "outm=" + out]
subprocess.call(map_cmd)

0

.
# Extracting Tat SL8 to Barcode Linkages

In [15]:
allLinkageData = []
#for the files in our out directory
for file in os.listdir(appRoot): 
    allAmplicons = []
    #ONLY take the files that end with fastq, there are also some stats files and hidden files that we don't want
    if file.endswith(".fastq"): 
        #using the Biopython parse we can read our fastq input
        fastq_parser = SeqIO.parse(appRoot + file, "fastq")
        for seq_record in fastq_parser: 
            #Take the current sequence
            sequence = str(seq_record.seq)
            allAmplicons.append(sequence)
            #only finding the amplicons that are long enough to be real      
    for amplicon in allAmplicons: 
        if (len(amplicon)) >= 451: 
            #we are making a list to append our barcode/SL8 variant pair to
            link = []
            #this will give us our animal name in one column
            link.append(file[0:6])
            #this will give us our timepoint in one column
            link.append(file.split("_")[1])
            #this will give us our rep number in one column
            link.append(file.split("_")[2][3:4])
            
            #this is necessary for r04103
            #amplicon5 = Seq.reverse_complement(amplicon[1:])
            
            #pulling out the barcode from r04103
            #tag = str(amplicon5[73:107])
            
            #pulling out the barcode from r10001
            tag = str((amplicon[1:])[72:106])
            
            link.append(tag)
            
            #now we are pulling out the Tat SL8 sequence for r04103
            #SL8 = str(amplicon5[339:363])
            
            #now we are pulling out the Tat SL8 sequence for r10001
            SL8 = str((amplicon[1:])[338:362])
            
            link.append(SL8)
            #now we want to translate SL8
            SL8AA = Seq.translate(SL8)
            #now we are adding the amino acid sequence of SL8 as the second part of the list
            link.append(SL8AA)
            allLinkageData.append(link)

df = pd.DataFrame(allLinkageData, columns=["Animal ID", "Time Point", 
                                           "Rep Number", "SIVmac239M Barcode", 
                                           "Tat SL8 Nucleotide Sequence",
                                           "Tat SL8 Amino Acid Sequence"])
linkage_df = df.groupby(["Animal ID", "Time Point", "Rep Number", 
                          "SIVmac239M Barcode", 
                          "Tat SL8 Nucleotide Sequence", 
                          "Tat SL8 Amino Acid Sequence"]).size().reset_index().rename(columns={0:'count'})
linked_sorted_df = linkage_df.sort_values(by=['count'], ascending=False)
print(linked_sorted_df)
linked_sorted_df.to_csv(appRoot + "r10001TatSL8toBarcodeLinkage.csv")           

     Animal ID Time Point Rep Number                  SIVmac239M Barcode  \
271     r10001      Day14          1  ACGCGCTTGCAAGCAATCTAGCGGTAGCCCGCGT   
844     r10001      Day21          1  ACGCGCTTGCAAGCAATCTAGCGGTAGCCCGCGT   
1853    r10001      Day28          1  ACGCGCTTGCAAGCAATCTAGCGGTAGCCCGCGT   
3101    r10001     Week12          1  ACGCGCTTGCAACTGAGCGGAGGGTAGCCCGCGT   
2446    r10001     Week10          1  ACGCGCTTGCAACTGAGCGGAGGGTAGCCCGCGT   
...        ...        ...        ...                                 ...   
2949    r10001     Week12          1  ACGCGCTTGCAACCAATCTAGCGGTAGCCCGCGT   
2950    r10001     Week12          1  ACGCGCTTGCAACCAATCTAGCGGTAGCCCGCGT   
2951    r10001     Week12          1  ACGCGCTTGCAACCAATCTAGCTGTAGCCCGCGT   
2952    r10001     Week12          1  ACGCGCTTGCAACCAATCTGAGGGTAGCCCGCGT   
0       r10001      Day14          1  AAACGCGCTTGCAACTGAGCGGAGGGTAGCCCGC   

     Tat SL8 Nucleotide Sequence Tat SL8 Amino Acid Sequence  count  
271     TCCACTCCA